## Convolutional Neural Network

Convolution is a mathematical operation that combines information from two sources to
produce a new set of information. Specifically, it applies a special matrix known as the
kernel to the input tensor to produce a set of matrices known as the feature maps. The kernel
can be applied to the input tensor using any of the popular algorithms.

The result of applying the previously-mentioned convolution algorithm is the feature
map which is the filtered version of the original tensor. For example, the feature map could
have only the outlines filtered from the original image. Hence, the kernel is also known as
the filter. For each kernel, you get a separate 2D feature map.

Depending on which features you want the network to learn, you have to
apply the appropriate filters to emphasize the required features. However,
with CNN, the model can automatically learn which kernels work best in
the convolution layer.

In [15]:
import os 
import numpy as np
import tensorflow as tf


In [16]:
from tensorflow.examples.tutorials.mnist import input_data

In [17]:
mnist=input_data.read_data_sets("data")

x_tr=mnist.train.images
y_tr=mnist.train.labels
x_ts=mnist.test.images
y_ts=mnist.test.labels


Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [18]:
tf.reset_default_graph()

In [19]:
n_classes=10
n_width=28
n_height=28
n_depth=1
n_input=n_width*n_height*n_depth

learning_rate=0.001
n_epochs=10
batch_size=256
n_batches=int(mnist.train.num_examples/batch_size)

In [20]:
n_input

784

In [21]:
y_tr.shape

(55000,)

In [22]:
tf.reset_default_graph()

In [23]:
x=tf.placeholder(tf.float32,shape=[None,n_input])
y=tf.placeholder(tf.int32,shape=[None])
i_x=tf.reshape(x,[-1,28,28,1])

with tf.name_scope("conv_layer1"):
    layer1_w=tf.Variable(tf.random_normal(shape=[4,4,n_depth,16],stddev=0.1),name="l1_w")
    layer1_b=tf.Variable(tf.random_normal(shape=[16],stddev=0.1),name="l1_b")
    layer1_conv=tf.nn.relu(tf.nn.conv2d(i_x,layer1_w,strides=[1,2,2,1],padding="SAME")+layer1_b)
    layer1_pool=tf.nn.max_pool(layer1_conv,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

with tf.name_scope("conv_layer2"):
    layer2_w=tf.Variable(tf.random_normal(shape=[4,4,16,32],stddev=0.1),name="l2_w")
    layer2_b=tf.Variable(tf.random_normal(shape=[32],stddev=0.1),name="l2_b")
    layer2_conv=tf.nn.relu(tf.nn.conv2d(layer1_pool,layer2_w,strides=[1,2,2,1],padding="SAME")+layer2_b)
    layer2_pool=tf.nn.max_pool(layer2_conv,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    
with tf.name_scope("fc_layer3"):
    h,w,d=layer2_pool.shape[1:]
    layer3_w=tf.Variable(tf.random_normal(shape=[int(h*w*d),124],stddev=0.01),name="l3_w")
    layer3_b=tf.Variable(tf.random_normal(shape=[124],stddev=0.01),name="l3_b")
    layer3_fc=tf.nn.relu(tf.matmul(tf.reshape(layer2_pool,[-1,h*w*d]),layer3_w)+layer3_b)

with tf.name_scope("logits_layer4"):
    layer4_w=tf.Variable(tf.random_normal(shape=[124,10],stddev=0.01),name="l4_w")
    layer4_b=tf.Variable(tf.random_normal(shape=[10],stddev=0.01),name="l4_b")
    logits=tf.matmul(layer3_fc,layer4_w)+layer4_b
    

In [24]:
losses=tf.losses.sparse_softmax_cross_entropy(labels=y,logits=logits)
train_op=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(losses)

In [25]:
correct=tf.nn.in_top_k(logits,y,1)
accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

In [26]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        n_steps=int(mnist.train.num_examples/batch_size)
        for step in range(n_steps):
            x_batch,y_batch=mnist.train.next_batch(batch_size)
           # print(x_batch,y_batch)
            sess.run(train_op ,feed_dict={x:x_batch,y:y_batch})
            sess.run(accuracy ,feed_dict={x:x_batch,y:y_batch})
            if step%200==0:
                v=sess.run([accuracy,losses],feed_dict={x:x_ts[:1000],y:y_ts[:1000]})
                print("Epoch : {} | Loss : {:.4f} | Accuracy : {:.4f}".format(epoch,v[0],v[1]))
                      

Epoch : 0 | Loss : 0.1070 | Accuracy : 2.3020
Epoch : 0 | Loss : 0.8620 | Accuracy : 0.4111
Epoch : 1 | Loss : 0.8990 | Accuracy : 0.3259
Epoch : 1 | Loss : 0.9350 | Accuracy : 0.2065
Epoch : 2 | Loss : 0.9390 | Accuracy : 0.1960
Epoch : 2 | Loss : 0.9420 | Accuracy : 0.1643
Epoch : 3 | Loss : 0.9520 | Accuracy : 0.1483
Epoch : 3 | Loss : 0.9620 | Accuracy : 0.1215
Epoch : 4 | Loss : 0.9570 | Accuracy : 0.1265
Epoch : 4 | Loss : 0.9710 | Accuracy : 0.0966
Epoch : 5 | Loss : 0.9620 | Accuracy : 0.1106
Epoch : 5 | Loss : 0.9610 | Accuracy : 0.1000
Epoch : 6 | Loss : 0.9630 | Accuracy : 0.0952
Epoch : 6 | Loss : 0.9700 | Accuracy : 0.0958
Epoch : 7 | Loss : 0.9620 | Accuracy : 0.0921
Epoch : 7 | Loss : 0.9630 | Accuracy : 0.0851
Epoch : 8 | Loss : 0.9730 | Accuracy : 0.0794
Epoch : 8 | Loss : 0.9680 | Accuracy : 0.0851
Epoch : 9 | Loss : 0.9710 | Accuracy : 0.0800
Epoch : 9 | Loss : 0.9690 | Accuracy : 0.0829


### Keras

In [27]:
import keras
from keras import layers
from keras import models
import time 
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [ ]:
tf.reset_default_graph()

In [28]:
model=models.Sequential()

model.add(layers.Conv2D(16,4,activation="relu",input_shape=(28,28,1),padding="SAME",strides=2
                        kernel_initializer='random_normal',bias_initializer='random_normal'))


model.add(layers.MaxPool2D(2,2,padding="SAME"))

model.add(layers.Conv2D(32,4,activation="relu",input_shape=(28,28,1),padding="SAME",
                        kernel_initializer='random_normal',bias_initializer='random_normal'))

model.add(layers.MaxPool2D(2,2,padding="SAME"))

model.add(layers.Flatten())
model.add(layers.Dense(124, kernel_initializer='random_normal',bias_initializer='random_normal',activation="relu"))
model.add(layers.Dense(10, kernel_initializer='random_normal',bias_initializer='random_normal',activation="softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        544       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              3212288   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10250     
Total para

In [29]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [30]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(time.time()),histogram_freq=0,  
          write_graph=True, write_images=True)
model.fit(x_tr.reshape(-1,28,28,1),y_tr,batch_size=64,epochs=10,callbacks=[tensorboard])

Epoch 1/10
55000/55000 [==============================] - 171s 3ms/step - loss: 0.1248 - acc: 0.9610
Epoch 2/10
55000/55000 [==============================] - 160s 3ms/step - loss: 0.0360 - acc: 0.9883
Epoch 3/10
55000/55000 [==============================] - 161s 3ms/step - loss: 0.0231 - acc: 0.9927
Epoch 4/10
55000/55000 [==============================] - 163s 3ms/step - loss: 0.0176 - acc: 0.9947
Epoch 5/10
55000/55000 [==============================] - 163s 3ms/step - loss: 0.0134 - acc: 0.9957
Epoch 6/10
55000/55000 [==============================] - 165s 3ms/step - loss: 0.0106 - acc: 0.9967
Epoch 7/10
55000/55000 [==============================] - 171s 3ms/step - loss: 0.0088 - acc: 0.9972
Epoch 8/10
55000/55000 [==============================] - 175s 3ms/step - loss: 0.0081 - acc: 0.9975
Epoch 9/10
55000/55000 [==============================] - 176s 3ms/step - loss: 0.0067 - acc: 0.9978
Epoch 10/10
55000/55000 [==============================] - 168s 3ms/step - loss: 0.0053 - a